In [1]:
from hetro_py.load_graph import load_graph
ogb_mag = load_graph('ogb-mag')
print(ogb_mag)
# apfpa
average_p_f = len(ogb_mag["paper","field_of_study"].edge_index[0]) / ogb_mag["paper"].num_nodes
average_f_p = len(ogb_mag["field_of_study","paper"].edge_index[0]) / ogb_mag["field_of_study"].num_nodes
average_p_a = len(ogb_mag["paper","author"].edge_index[0]) / ogb_mag["paper"].num_nodes
print(average_p_f)
apf = len(ogb_mag["author","paper"].edge_index[0]) * average_p_f
apfpa = len(ogb_mag["author","paper"].edge_index[0]) * average_p_f * average_f_p * average_p_a
print("apf: ",apf * 4 /1024 / 1024," MB")
print("apfpa: ",apfpa *4 / 1024 / 1024," MB")
# the csr 
csr_data = ogb_mag.csr(store = True)
print(csr_data)

/home/sjq/.conda/envs/cuda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


HeteroData(
  paper={
    x=[736389, 128],
    year=[736389],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ num_nodes=1134649 },
  institution={ num_nodes=8740 },
  field_of_study={ num_nodes=59965 },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)
10.191730186083714
apf:  277.81157959553127  MB
apfpa:  337398.2079521468  MB
({('author', 'affiliated_with', 'institution'): tensor([      0,       1,       2,  ..., 1043996, 1043997, 1043998]), ('author', 'writes', 'paper'): tensor([      0,       4,       7,  ..., 7145654, 7145658, 7145660

In [6]:
print(csr_data[0]["author","affiliated_with","institution"])
print(csr_data[1]["author","affiliated_with","institution"])
print(csr_data[2]["author","affiliated_with","institution"])
print(ogb_mag["author","affiliated_with","institution"])

tensor([      0,       1,       2,  ..., 1043996, 1043997, 1043998])
tensor([ 845,  996, 3197,  ..., 5189, 4668, 4668])
tensor([      0,       1,       2,  ..., 1043995, 1043996, 1043997])
{'edge_index': tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
        [    845,     996,    3197,  ...,    5189,    4668,    4668]]), 'adj': SparseTensor(row=tensor([      0,       1,       2,  ..., 1134645, 1134647, 1134648]),
             col=tensor([ 845,  996, 3197,  ..., 5189, 4668, 4668]),
             size=(1134649, 8740), nnz=1043998, density=0.01%)}


In [34]:
print(csr_data[0]["paper","cites","paper"])
print(csr_data[1]["paper","cites","paper"])
print(csr_data[2]["paper","cites","paper"])
print(ogb_mag["paper","cites","paper"])
print(ogb_mag["paper","paper"].adj.dim())
start = "paper"
end = "paper"
node =0
print(ogb_mag[start,end].adj[node])
print(ogb_mag["paper","paper"].adj[0])
for col in ogb_mag["paper","paper"].adj[node].cols():
    print(col)
print(ogb_mag["paper","paper"].adj.size(0))
print(ogb_mag["paper","paper"].adj.size(1))
print(ogb_mag["author","paper"].adj.size(0))
print(ogb_mag["author","paper"].adj.size(1))
from torch_geometric.typing import OptTensor, SparseTensor

tensor([       0,        6,        7,  ..., 10792606, 10792612, 10792672])
tensor([    88,  27449, 121051,  ..., 707740, 732008, 732389])
tensor([       0,        1,        2,  ..., 10792669, 10792670, 10792671])
{'edge_index': tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
        [    88,  27449, 121051,  ..., 707740, 732008, 732389]]), 'adj': SparseTensor(row=tensor([     0,      0,      0,  ..., 736388, 736388, 736388]),
             col=tensor([    88,  27449, 121051,  ..., 707740, 732008, 732389]),
             size=(736389, 736389), nnz=10792672, density=0.00%)}
2
SparseTensor(row=tensor([0, 0, 0, 0, 0, 0]),
             col=tensor([    88,  27449, 121051, 151667, 308499, 368076]),
             size=(1, 736389), nnz=6, density=0.00%)
SparseTensor(row=tensor([0, 0, 0, 0, 0, 0]),
             col=tensor([    88,  27449, 121051, 151667, 308499, 368076]),
             size=(1, 736389), nnz=6, density=0.00%)


AttributeError: 'SparseTensor' object has no attribute 'cols'

In [29]:
from typing import *
def get_metapath(data,start_node,metapath)->int:
    start = metapath[0]
    end = metapath[1]
    neighbors = data[(start,end)].adj[start_node]
    if len(metapath) == 2:
        return len(neighbors)
    else:
        total=0
        for neighbor in neighbors:
            child_nodes=get_metapath(data,neighbor,metapath[1:])
            total += child_nodes
        return total

def calculate_memory(data,metapath)->int:
    start = metapath[0]
    end = metapath[1]
    rows = data[start,end].adj.size(0)
    total = 0
    for i in range(rows):
        total += get_metapath(data,i,metapath)
    return total


In [30]:
memory = calculate_memory(ogb_mag,["author","paper","author"])
print(memory)

SyntaxError: None (<string>)